# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [34]:
import scipy.io as so  # Importing the scipy.io module for loading data from .mat files.
import numpy as np  # Importing the NumPy library for numerical operations.

data = so.loadmat('data/mammography.mat')  # Loading data from a MATLAB .mat file located in the 'data' directory.

np.random.seed(0)  # Setting the random seed to 0 for reproducibility of random operations.

shuffle_index = np.arange(0, len(data['X']))  # Creating an array of indices from 0 to the length of 'X' in the loaded data.

np.random.shuffle(shuffle_index)  # Randomly shuffling these indices to randomize the order of the data.

X = data['X'][shuffle_index, :]  # Rearranging 'X' data in the loaded dataset according to the shuffled indices.
                                  # This step randomizes the rows of 'X' without changing the data in each row.

Y = data['y'][shuffle_index, :].flatten()  # Similarly rearranging 'y' data and then flattening it.
                                           # 'flatten()' converts 'y' from a 2D array to a 1D array for easier handling.


In [35]:

# Part 1
train_size = int(0.6 * len(X))  # Calculating the size of the training set as 60% of the total dataset.
test_size = int(0.15 * len(X))  # Calculating the size of the test set as 15% of the total dataset.
valid_size = int(len(X) - train_size - test_size)  # Calculating the size of the validation set. It's the remainder of the dataset after allocating to training and test sets.

X_train = X[:train_size]  # Slicing the first 'train_size' number of instances from X for the training set.
X_test = X[train_size:test_size + train_size]  # Slicing the next 'test_size' number of instances from X for the test set.
X_valid = X[test_size + train_size:]  # Slicing the remaining instances from X for the validation set.

Y_train = Y[:train_size]  # Similarly slicing Y for the training set labels.
Y_test = Y[train_size:test_size + train_size]  # Slicing Y for the test set labels.
Y_valid = Y[test_size + train_size:]  # Slicing Y for the validation set labels.

# Verifying the shapes of the split datasets.
# The expected shapes are for the feature matrices (X) to be (6709, 6), (1677, 6), (2797, 6) 
# and for the label vectors (Y) to be (6709,), (1677,), (2797,)
X_train.shape, X_test.shape, X_valid.shape, Y_train.shape, Y_test.shape, Y_valid.shape


((6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,))

In [36]:
from sklearn.linear_model import LogisticRegression  # Importing the Logistic Regression model from scikit-learn.
from sklearn.preprocessing import StandardScaler    # Importing the StandardScaler for feature scaling.
from sklearn.pipeline import Pipeline                # Importing Pipeline to sequentially apply a list of transforms and a final estimator.

# Part 2
# Training a logistic regression model using the training data (X_train, Y_train).

# Creating a pipeline with two steps:
# 1. 'scaler': StandardScaler() - This will standardize features by removing the mean and scaling to unit variance.
#    Standardizing the features is often important for many machine learning algorithms, including logistic regression,
#    because it can lead to better performance and more stable models.
# 2. 'logistic_regression': LogisticRegression() - This adds the logistic regression model to the pipeline.
#    Logistic regression is a good choice for binary classification problems.
model = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic_regression', LogisticRegression())
])

# Fitting the model to the training data.
# The pipeline first applies the standard scaler to the data and then fits the logistic regression model.
model.fit(X_train, Y_train)



Pipeline(steps=[('scaler', StandardScaler()),
                ('logistic_regression', LogisticRegression())])

In [37]:
import sys
# Adding a custom path to the system path.
# This is necessary to import modules from a non-standard directory.
sys.path.append('/path/to/utils_module')

# Import the custom utility function.
from Utils import classification_report_interval

# Defining the true labels and predicted labels for the test set.
y_true = Y_test  # True labels from the test set.
y_pred = model.predict(X_test)  # Predicted labels obtained by applying the trained model to the test set.

# Compute precision and recall intervals using the classification_report_interval function.
# The function is expected to return confidence intervals for precision and recall metrics.
report = classification_report_interval(
    y_true,
    y_pred,
    labels=None,  # If 'None', the labels are inferred from the true labels (y_true).
    alpha = 0.01,  # Significance level for the confidence intervals. Here, it's set to 1% (0.01).
    union_bound_correction=True  # Indicates whether to apply union bound correction for the intervals.
)

# Printing the report which contains the precision and recall intervals.
print(report)


            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.95,1.00]
                 1  0.70 : [0.28,1.00] 0.36 : [0.06,0.66]

          accuracy                                        0.98 : [0.94,1.00]



In [38]:

# Part 3
import sys
sys.path.append('/path/to/utils_module')

# Compute precision and recall on the test set using 
from Utils import classification_report_interval
y_true = Y_test
y_pred =model.predict(X_test)

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)
report =classification_report_interval(
    y_true,
    y_pred,
    labels=None,
    alpha = 0.01,
    union_bound_correction=True
)


precision0 = (0.94, 1.00)  # Interval for precision of class 0
recall0 = (0.96, 1.00)     # Interval for recall of class 0
precision1 = (0.33, 1.00)  # Interval for precision of class 1
recall1 = (0.09, 0.62)

# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

In [39]:

# Part 4
def cost(model, threshold, X, Y):
    # Get the probability of the positive class from the model's predictions.
    pred_proba = model.predict_proba(X)[:, 1]

    # Convert these probabilities to binary predictions based on the specified threshold.
    # Predictions are 1 if the probability is greater than or equal to the threshold, 0 otherwise.
    predictions = (pred_proba >= threshold) * 1
    
    # Define the costs associated with different prediction outcomes.
    cost_false_negative = 30  # Cost incurred for each false negative prediction.
    cost_false_positive = 5   # Cost incurred for each false positive prediction.
    # Costs for true positives and true negatives are set to 0, assuming no cost for correct predictions.

    # Calculate the number of false negatives and false positives.
    false_negatives = ((predictions == 0) & (Y == 1)).sum()  # Count instances where the model predicts 0 (negative) but the actual label is 1 (positive).
    false_positives = ((predictions == 1) & (Y == 0)).sum()  # Count instances where the model predicts 1 (positive) but the actual label is 0 (negative).

    # Calculate the total cost based on the number of false negatives and positives and their respective costs.
    total_cost = (false_negatives * cost_false_negative) + (false_positives * cost_false_positive)

    # Compute the average cost per instance in the dataset.
    average_cost = total_cost / len(Y)

    return average_cost


In [40]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set

naive_cost =cost(model,1,X_test,Y_test)#A threshold of 1 means that the classifier will only predict the positive class (e.g., 'spam' in a spam detection scenario) if it is 100% certain.
naive_cost

0.6976744186046512

In [41]:

# Part 5
# Generate 100 evenly spaced values between 0 and 1.
thresholds = np.linspace(0, 1, 100)

# Calculate the cost for each threshold value.
# This is done by applying the 'cost' function to the model's predictions on the test set (X_test, Y_test)
# for each threshold in the 'thresholds' array.
costs = [cost(model, t, X_test, Y_test) for t in thresholds]

# Find the index of the threshold that results in the lowest cost.
# 'np.argmin(costs)' returns the index of the minimum value in the 'costs' array.
optimal_index = np.argmin(costs)

# Retrieve the optimal threshold value.
# This is the threshold corresponding to the lowest cost.
optimal_threshold = thresholds[optimal_index]

# Calculate the cost at this optimal threshold.
# This gives us the minimum average cost achievable by the model on the test data, using the best threshold.
cost_at_optimal_threshold = costs[optimal_index]


0.11111111111111112

In [48]:
import math
from Utils import epsilon_bounded
# Part 6
n= len(Y_valid)
cost_at_optimal_threshold_validation =cost(model,optimal_threshold,X_valid,Y_valid)
# Report the cost interval as a tuple cost_interval = (a,b)
bound = epsilon_bounded(n,30,0.05)

cost_interval =(max(cost_at_optimal_threshold_validation-bound,0),cost_at_optimal_threshold_validation+bound)

# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)
cost_interval

(0, 1.0617676271684535)

In [54]:
C = 30 * (1 - (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)) * Y_valid + 5 * (1 - Y_valid) * (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)
# Part 7

variance_of_C = np.var(C)
variance_of_C

6.198613637958541

In [58]:
from Utils import bennett_epsilon  # Importing the bennett_epsilon function from a custom utility module.
import scipy.optimize as so  # Importing the scipy.optimize module for optimization tasks.
import numpy as np  # Importing the NumPy library for numerical operations.

# Part 7
n = len(Y_valid)  # The number of data points in the validation set.
alpha = 0.05  # Significance level set to 5%, which is typical for statistical tests.

b = 30  # This might represent the maximum possible deviation per observation, used in Bennett's inequality.

# Assuming 'variance_of_C' is defined earlier in the code. It's the variance of the random variable C.
sigma = np.sqrt(variance_of_C)  # Standard deviation is the square root of the variance.

# Calculating the Bennett epsilon value, which is used to establish the confidence interval.
# This function likely implements Bennett's inequality to calculate the bound based on provided parameters.
epsilon_bennett = bennett_epsilon(n, 30, sigma, 0.05)

# Calculating the 95% confidence interval for the random variable C.
# The interval is centered around the mean of C, with the width determined by epsilon_bennett.
interval_of_C = (np.mean(C) - epsilon_bennett, np.mean(C) + epsilon_bennett)

# Assertions to ensure that the interval is a tuple and has two elements (lower and upper bounds).
assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

# The interval_of_C is the final result, representing the confidence interval.
interval_of_C


Numerical error -1.1102230246251565e-16


(0.15091432947852831, 0.431852921147142)